In [29]:
#!pip install git+https://github.com/mlrun/mlrun@fix-remote-nuclio

### Subscribe to camera feeeds 
This function subscribes to a video feed and sends frames to 
an Iguazio stream

In [30]:
import nuclio

In [45]:
%run set_env.ipynb

In [32]:
# nuclio: start-code

In [ ]:
import cv2
from cv2 import VideoWriter_fourcc, VideoWriter
import sys
import os
import requests
import base64
import numpy as np

In [107]:
def stream_frame_write(context,frame,shard):
    url = "http://v3io-webapi:8081/%s/%s/"% (os.getenv('IGZ_CONTAINER'),os.getenv('TAGGED_VIDEO_STREAM'))
    headers = {
            "Content-Type": "application/json",
            "X-v3io-function": "PutRecords",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
          }
    data = base64.b64encode(frame)
    payload = {
            "Records": [
                {
                    "Data":  data.decode('utf-8'),
                    "ShardId" : shard
                    }
                ]
          }
    r = requests.post(url, headers=headers,json=payload)   
    return r.text

In [108]:
def handler(context,event):
    shardId = event.shard_id
    event_data=event.body
    
    # Load the cascade
    face_cascade = cv2.CascadeClassifier(os.getenv('CASCADE_CLASSIFIER'))

    # To capture video from webcam.
    img_raw = event_data
    #open("/User/tmp/img.jpg","wb").write(event_data)
    img = np.asarray(bytearray(img_raw), dtype="uint8")
    #img = cv2.imdecode(img,cv2.COLOR_BGR2RGBA)
    img = cv2.imdecode(img,cv2.COLOR_BGR2GRAY)
    #gray = cv2.imdecode(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(img, 1.1, 4)
    
    # Draw the rectangle around each face
    face_count=0
    for (x, y, w, h) in faces:
        face_count += 1
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
    #context.logger.info("Face Count: %s"% face_count )
    if face_count > 0 :
        ret, buffer = cv2.imencode('.jpg', img)
        try:
            r = stream_frame_write(context,buffer.tobytes(),shardId)
        except:
            context.logger.error("Failed to save frame for shard %s"% shardId)
        #save_count_and_image(face_count,'105lf')



In [109]:
# nuclio: end-code

In [110]:
import os

In [111]:
# nuclio: ignore
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function, mount_v3io, mlconf
import os

fn = code_to_function('face-recognition', kind='nuclio')

In [112]:
# set the API/trigger, attach the home dir to the function
fn.with_http(workers=1).apply(mount_v3io())
fn.spec.build.commands = ['pip install opencv-python v3io requests numpy']
fn.spec.build.base_image = 'mlrun/mlrun'
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
fn.set_env('V3IO_ACCESS_KEY',os.getenv('V3IO_ACCESS_KEY'))
fn.set_env('IGZ_CONTAINER',os.getenv('IGZ_CONTAINER'))
fn.set_env('TAGGED_VIDEO_STREAM',os.getenv('TAGGED_VIDEO_STREAM'))
fn.set_env('CAMERA_LIST_TBL',os.getenv('CAMERA_LIST_TBL'))
fn.set_env('CASCADE_CLASSIFIER',os.getenv('CASCADE_CLASSIFIER'))
trigger_spec={
      'kind': 'v3ioStream',
      'url' : "http://v3io-webapi:8081/%s/%s@processorgrp"% (os.getenv('IGZ_CONTAINER'),os.getenv('RAW_VIDEO_STREAM')),
    "password": os.getenv('V3IO_ACCESS_KEY'),  
    'attributes': {"pollingIntervalMs": 500,
        "seekTo": "latest",
        "readBatchSize": 100,
        "partitions": "0-100",               
                     
      }
    }
fn.add_trigger('image-proc',trigger_spec)

fn.apply(mount_v3io())

In [113]:
# nuclio: ignore
addr = fn.deploy(project='objectrecognition')

[mlrun] 2020-07-05 16:32:32,538 deploy started
[nuclio] 2020-07-05 16:32:38,676 (info) Build complete
[nuclio] 2020-07-05 16:32:42,750 (info) Function deploy complete
[nuclio] 2020-07-05 16:32:42,757 done updating objectrecognition-face-recognition, function address: 3.128.161.62:32711
